In [1]:
import pandas as pd
import numpy as np

### Aggregations for CNA at gene level

In [2]:
df_cna_long = pd.read_csv('UCEC_discovery_WGS_CNA_gene_level_THRESHOLD__with_locus.csv', low_memory=False)
df_cna_long

,id,gene,cna,gene_name,chr,karyotype_band,chr_arm,chr_region,chr_region_band
0,S001,FAM138A|chr1,0,FAM138A,1,p36.33,1p,1p3,1p36
1,S002,FAM138A|chr1,0,FAM138A,1,p36.33,1p,1p3,1p36
2,S003,FAM138A|chr1,0,FAM138A,1,p36.33,1p,1p3,1p36
3,S005,FAM138A|chr1,0,FAM138A,1,p36.33,1p,1p3,1p36
4,S006,FAM138A|chr1,0,FAM138A,1,p36.33,1p,1p3,1p36
...,...,...,...,...,...,...,...,...,...
2398175,S099,TTTY3B,0,TTTY3B,Y,q11.23,Yq,Yq1,Yq11
2398176,S100,TTTY3B,0,TTTY3B,Y,q11.23,Yq,Yq1,Yq11
2398177,S101,TTTY3B,0,TTTY3B,Y,q11.23,Yq,Yq1,Yq11
2398178,S102,TTTY3B,0,TTTY3B,Y,q11.23,Yq,Yq1,Yq11


In [3]:
group_by_column = 'chr_arm'
aggs = [
    'min',
    'max',
    'count', 
    ('count_nonzero', lambda x: np.count_nonzero(x)),
    ('abs_sum', lambda x: x.abs().sum()),
    ('neg_sum', lambda x: np.minimum(x, 0).sum()),
    ('pos_sum', lambda x: np.maximum(x, 0).sum()),
]
agg_names = [s if isinstance(s, str) else s[0] for s in aggs]

df_cna_long_agg = df_cna_long.groupby(by=['id', group_by_column])['cna'].agg(aggs)
df_cna_long_agg.reset_index().head(25)
df_cna_long_agg

min  max  count  count_nonzero  abs_sum  neg_sum  pos_sum
id   chr_arm                                                           
S001 10p        0    0    136              0        0        0        0
     10q       -2    0    607              2        4       -4        0
     11p        0    0    364              0        0        0        0
     11q        0    0    784              0        0        0        0
     12p        0    0    253              0        0        0        0
...           ...  ...    ...            ...      ...      ...      ...
S103 9q         0    0    521              0        0        0        0
     Xp         0    0    347              0        0        0        0
     Xq         0    0    324              0        0        0        0
     Yp         0    0     43              0        0        0        0
     Yq         0    0     47              0        0        0        0

[4180 rows x 7 columns]

In [4]:
df_cna_long_agg.sort_values(by=['abs_sum'], ascending=False).head(25)

,,min,max,count,count_nonzero,abs_sum,neg_sum,pos_sum
id,chr_arm,,,,,,,
S073,1q,0,2,793,788,1576,0,1576
S062,1q,0,2,793,651,1302,0,1302
S079,1p,-2,2,1015,888,1282,-640,642
S091,1q,0,2,793,787,1271,0,1271
S100,1p,-1,2,1015,1015,1026,-389,637
S066,1q,-2,2,793,482,964,-18,946
S031,2q,0,2,668,666,926,0,926
S021,1q,0,2,793,790,925,0,925
S031,11q,-1,2,784,783,919,-646,273


In [5]:
df_cna_long_agg_pivot = df_cna_long_agg.reset_index().pivot_table(index='id', columns=group_by_column, values=agg_names)
df_cna_long_agg_pivot.head()

abs_sum                                                          \
chr_arm     10p    10q    11p    11q    12p    12q    13q    14q    15q   
id                                                                        
S001        0.0    4.0    0.0    0.0    0.0    0.0    0.0    0.0    2.0   
S002        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
S003        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
S005      139.0  605.0    0.0    2.0    0.0    0.0    2.0    2.0   18.0   
S006      137.0  622.0  360.0  784.0  253.0  698.0  434.0  557.0  575.0   

                ... pos_sum                                                \
chr_arm    16p  ...      7p    7q     8p     8q    9p    9q     Xp     Xq   
id              ...                                                         
S001       0.0  ...     0.0   0.0    0.0    0.0   0.0   0.0    0.0    0.0   
S002       0.0  ...     0.0   0.0    0.0    0.0   0.0   0.0    0.0    0.0   
S003       0.0  ...     0.0   0.0    0.0    0.0   0.0   0.0    0.0    0.0   
S005      16.0  ...     4.0   7.0  153.0  420.0   0.0   0.0    0.0    0.0   
S006     420.0  ...   234.0  50.0   32.0  661.0  85.0  27.0  332.0  326.0   

                   
chr_arm   Yp   Yq  
id                 
S001     0.0  0.0  
S002     0.0  0.0  
S003     0.0  0.0  
S005     0.0  0.0  
S006     0.0  0.0  

[5 rows x 308 columns]

In [6]:
column_names = [f'{pair[1]}_{pair[0]}_cna' for pair in df_cna_long_agg_pivot.columns]
df_cna_long_agg_pivot.columns = column_names
df_cna_long_agg_pivot = df_cna_long_agg_pivot.reset_index()
df_cna_long_agg_pivot

,id,10p_abs_sum_cna,10q_abs_sum_cna,11p_abs_sum_cna,11q_abs_sum_cna,12p_abs_sum_cna,12q_abs_sum_cna,13q_abs_sum_cna,14q_abs_sum_cna,15q_abs_sum_cna,...,7p_pos_sum_cna,7q_pos_sum_cna,8p_pos_sum_cna,8q_pos_sum_cna,9p_pos_sum_cna,9q_pos_sum_cna,Xp_pos_sum_cna,Xq_pos_sum_cna,Yp_pos_sum_cna,Yq_pos_sum_cna
0,S001,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,S002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,S003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,S005,139.0,605.0,0.0,2.0,0.0,0.0,2.0,2.0,18.0,...,4.0,7.0,153.0,420.0,0.0,0.0,0.0,0.0,0.0,0.0
4,S006,137.0,622.0,360.0,784.0,253.0,698.0,434.0,557.0,575.0,...,234.0,50.0,32.0,661.0,85.0,27.0,332.0,326.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,S099,0.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,283.0,524.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0
91,S100,2.0,9.0,131.0,53.0,4.0,44.0,416.0,70.0,26.0,...,17.0,621.0,49.0,786.0,7.0,1.0,152.0,340.0,0.0,0.0
92,S101,0.0,10.0,18.0,0.0,20.0,114.0,2.0,12.0,38.0,...,0.0,0.0,0.0,2.0,14.0,12.0,0.0,0.0,0.0,2.0
93,S102,0.0,40.0,74.0,459.0,16.0,138.0,39.0,290.0,43.0,...,60.0,98.0,32.0,132.0,0.0,0.0,110.0,0.0,0.0,0.0


Here, we generalize the pivot-aggregation for any table in long format.

In [7]:
def pivot_aggregations(df, id_column, group_by_column, value_column, aggs):
    '''
    Compute the given aggregations 'aggs' over the values of 'value_column' for 
    the groups defined by the column 'group_by_column'.
    The resulting DataFrame is pivoted with the 'id_column' as index.

    aggs - list of (1) strings (containing standard pandas function names) or
      (2) tuples with the aggregation function and the name of the resulting column.
    '''
    assert id_column in df.columns, f'{id_column} not in columns'
    assert group_by_column in df.columns, f'{group_by_column} not in columns'
    assert value_column in df.columns, f'{value_column} not in columns'

    df_group_by_agg = df.groupby(by=[id_column, group_by_column])[value_column].agg(aggs)

    agg_names = [s if isinstance(s, str) else s[0] for s in aggs]
    df_pivot = df_group_by_agg.reset_index().pivot_table(index=id_column, columns=group_by_column, values=agg_names)
    
    column_names = [f'{pair[1]}_{pair[0]}_{value_column}' for pair in df_pivot.columns]
    df_pivot.columns = column_names
    df_pivot.reset_index(inplace=True)
    
    return df_pivot

In [8]:
pivot_aggregations(df_cna_long, id_column='id', group_by_column='chr_arm', value_column='cna', aggs=aggs)

,id,10p_abs_sum_cna,10q_abs_sum_cna,11p_abs_sum_cna,11q_abs_sum_cna,12p_abs_sum_cna,12q_abs_sum_cna,13q_abs_sum_cna,14q_abs_sum_cna,15q_abs_sum_cna,...,7p_pos_sum_cna,7q_pos_sum_cna,8p_pos_sum_cna,8q_pos_sum_cna,9p_pos_sum_cna,9q_pos_sum_cna,Xp_pos_sum_cna,Xq_pos_sum_cna,Yp_pos_sum_cna,Yq_pos_sum_cna
0,S001,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,S002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,S003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,S005,139.0,605.0,0.0,2.0,0.0,0.0,2.0,2.0,18.0,...,4.0,7.0,153.0,420.0,0.0,0.0,0.0,0.0,0.0,0.0
4,S006,137.0,622.0,360.0,784.0,253.0,698.0,434.0,557.0,575.0,...,234.0,50.0,32.0,661.0,85.0,27.0,332.0,326.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,S099,0.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,283.0,524.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0
91,S100,2.0,9.0,131.0,53.0,4.0,44.0,416.0,70.0,26.0,...,17.0,621.0,49.0,786.0,7.0,1.0,152.0,340.0,0.0,0.0
92,S101,0.0,10.0,18.0,0.0,20.0,114.0,2.0,12.0,38.0,...,0.0,0.0,0.0,2.0,14.0,12.0,0.0,0.0,0.0,2.0
93,S102,0.0,40.0,74.0,459.0,16.0,138.0,39.0,290.0,43.0,...,60.0,98.0,32.0,132.0,0.0,0.0,110.0,0.0,0.0,0.0


In [9]:
df_cna_aggs = pivot_aggregations(df_cna_long, id_column='id', group_by_column='chr_arm', value_column='cna', aggs=aggs)
df_cna_aggs.to_csv('./data/preprocessed/UCEC_discovery_WGS_CNA_gene_level_THRESHOLD__locus_gene_arm_aggregation.csv', index=False)
print(df_cna_aggs.shape)

df_cna_aggs = pivot_aggregations(df_cna_long, id_column='id', group_by_column='chr_region', value_column='cna', aggs=aggs)
df_cna_aggs.to_csv('./data/preprocessed/UCEC_discovery_WGS_CNA_gene_level_THRESHOLD__locus_gene_arm_region_aggregation.csv', index=False)
print(df_cna_aggs.shape)

df_cna_aggs = pivot_aggregations(df_cna_long, id_column='id', group_by_column='chr_region_band', value_column='cna', aggs=aggs)
df_cna_aggs.to_csv('./data/preprocessed/UCEC_discovery_WGS_CNA_gene_level_THRESHOLD__locus_gene_region_band_aggregation.csv', index=False)
print(df_cna_aggs.shape)

(95, 309)
(95, 561)
(95, 1499)


### Aggregations for somatic mutations

In [10]:
df_sommut_long = pd.read_csv('UCEC_discovery_somatic_mutation_gene_level__with_locus.csv', low_memory=False)
df_sommut_long

,id,gene,sommut,chr,karyotype_band,chr_arm,chr_region,chr_region_band
0,S001,A1BG,0,19,q13.43,19q,19q1,19q13
1,S002,A1BG,0,19,q13.43,19q,19q1,19q13
2,S003,A1BG,0,19,q13.43,19q,19q1,19q13
3,S005,A1BG,0,19,q13.43,19q,19q1,19q13
4,S006,A1BG,0,19,q13.43,19q,19q1,19q13
...,...,...,...,...,...,...,...,...
1411600,S099,ZZZ3,0,1,p31.1,1p,1p3,1p31
1411601,S100,ZZZ3,0,1,p31.1,1p,1p3,1p31
1411602,S101,ZZZ3,0,1,p31.1,1p,1p3,1p31
1411603,S102,ZZZ3,0,1,p31.1,1p,1p3,1p31


In [11]:
pivot_aggregations(df_sommut_long, id_column='id', group_by_column='chr_arm', value_column='sommut', aggs=aggs)

,id,10p_abs_sum_sommut,10q_abs_sum_sommut,11p_abs_sum_sommut,11q_abs_sum_sommut,12p_abs_sum_sommut,12q_abs_sum_sommut,13q_abs_sum_sommut,14q_abs_sum_sommut,15q_abs_sum_sommut,...,6p_pos_sum_sommut,6q_pos_sum_sommut,7p_pos_sum_sommut,7q_pos_sum_sommut,8p_pos_sum_sommut,8q_pos_sum_sommut,9p_pos_sum_sommut,9q_pos_sum_sommut,Xp_pos_sum_sommut,Xq_pos_sum_sommut
0,S001,4.0,18.0,6.0,19.0,7.0,22.0,13.0,25.0,16.0,...,9.0,12.0,5.0,25.0,6.0,8.0,6.0,15.0,10.0,12.0
1,S002,2.0,13.0,4.0,12.0,3.0,10.0,3.0,6.0,7.0,...,2.0,6.0,1.0,4.0,3.0,7.0,2.0,10.0,4.0,3.0
2,S003,1.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,0.0
3,S005,1.0,0.0,0.0,2.0,0.0,2.0,1.0,3.0,1.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,0.0
4,S006,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,S099,18.0,66.0,38.0,86.0,27.0,91.0,32.0,61.0,60.0,...,30.0,32.0,45.0,75.0,13.0,43.0,22.0,72.0,42.0,31.0
91,S100,0.0,0.0,1.0,2.0,2.0,1.0,0.0,1.0,2.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,S101,1.0,0.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0
93,S102,0.0,2.0,2.0,1.0,0.0,3.0,0.0,1.0,2.0,...,0.0,1.0,0.0,2.0,1.0,1.0,0.0,1.0,1.0,1.0


In [12]:
df_sommut_aggs = pivot_aggregations(df_sommut_long, id_column='id', group_by_column='chr_arm', value_column='sommut', aggs=aggs)
df_sommut_aggs.to_csv('./data/preprocessed/UCEC_discovery_somatic_mutation_gene_level__locus_gene_arm_aggregation.csv', index=False)
print(df_sommut_aggs.shape)

df_sommut_aggs = pivot_aggregations(df_sommut_long, id_column='id', group_by_column='chr_region', value_column='sommut', aggs=aggs)
df_sommut_aggs.to_csv('./data/preprocessed/UCEC_discovery_somatic_mutation_gene_level__locus_gene_region_aggregation.csv', index=False)
print(df_sommut_aggs.shape)

df_sommut_aggs = pivot_aggregations(df_sommut_long, id_column='id', group_by_column='chr_region_band', value_column='sommut', aggs=aggs)
df_sommut_aggs.to_csv('./data/preprocessed/UCEC_discovery_somatic_mutation_gene_level__locus_gene_region_band_aggregation.csv', index=False)
print(df_sommut_aggs.shape)

(95, 295)
(95, 547)
(95, 1478)
